In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

2025-02-15 02:48:28.391950: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-15 02:48:28.393187: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-15 02:48:28.397083: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-15 02:48:28.413773: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739580508.441145   13653 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739580508.44

In [3]:
print(tf.__version__)

2.18.0


In [4]:
import tensorflowjs as tfjs
print(tfjs.__version__)

4.22.0


In [5]:
dataset_path = '/mnt/d/Career/nano courses/opencv/done_data/gray/chess_pieces_dataset/'
train_path = dataset_path + 'train/'
test_path = dataset_path + 'test/'

In [6]:
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,  # 20% for validation
    horizontal_flip=False,
)

train_gen = datagen.flow_from_directory(
    train_path,
    target_size=(44, 44),
    color_mode="grayscale",
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_gen = datagen.flow_from_directory(
    train_path,
    target_size=(44, 44),
    color_mode="grayscale",
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 10164 images belonging to 7 classes.
Found 2540 images belonging to 7 classes.


In [7]:
def count_images_per_class(dataset_path):
    class_counts = {}
    for class_name in os.listdir(dataset_path):
        print(class_name)
        class_dir = os.path.join(dataset_path, class_name)
        if not os.path.isdir(class_dir):
            continue  # Skip if not a directory

        num_images = len([f for f in os.listdir(class_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
        class_counts[class_name] = num_images

    return class_counts

print("\n✅ Number of images per class AFTER augmentation:")
after_counts = count_images_per_class(train_path)
for class_name, count in after_counts.items():
    print(f"  {class_name}: {count} images")


✅ Number of images per class AFTER augmentation:
b
e
k
n
p
q
r
  b: 1856 images
  e: 1800 images
  k: 1728 images
  n: 1800 images
  p: 1800 images
  q: 1800 images
  r: 1920 images


In [8]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(44, 44, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(7, activation='softmax')  # Assuming 6 pieces + 1 empty
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

/home/ghaly/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-02-15 02:49:03.344184: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 42, 42, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 21, 21, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 19, 19, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 9, 9, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 7, 7, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 6272)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       802,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 7)              │           903 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 896,519 (3.42 MB)

 Trainable params: 896,519 (3.42 MB)

 Non-trainable params: 0 (0.00 B)

In [1]:
import PIL; print(PIL.__version__)

11.1.0


In [11]:
num_epochs = 2
for epoch in range(1, num_epochs + 1):
    print(f"\nEpoch {epoch}/{num_epochs}")
    history = model.fit(train_gen, validation_data=val_gen, epochs=1, verbose=1)

    train_loss, train_acc = history.history["loss"][0], history.history["accuracy"][0]
    val_loss, val_acc = history.history["val_loss"][0], history.history["val_accuracy"][0]

    print(f"Train Loss: {train_loss:.4f} | Train Accuracy: {train_acc:.4f}")
    print(f"Val Loss: {val_loss:.4f} | Val Accuracy: {val_acc:.4f}")

print("\nTraining complete! 🎉")


Epoch 1/2
318/318 ━━━━━━━━━━━━━━━━━━━━ 85s 267ms/step - accuracy: 0.9965 - loss: 0.0106 - val_accuracy: 0.9965 - val_loss: 0.0899
Train Loss: 0.0077 | Train Accuracy: 0.9976
Val Loss: 0.0899 | Val Accuracy: 0.9965

Epoch 2/2
318/318 ━━━━━━━━━━━━━━━━━━━━ 85s 266ms/step - accuracy: 0.9997 - loss: 0.0010 - val_accuracy: 0.9953 - val_loss: 0.1003
Train Loss: 0.0012 | Train Accuracy: 0.9996
Val Loss: 0.1003 | Val Accuracy: 0.9953

Training complete! 🎉


In [12]:
test_gen = datagen.flow_from_directory(
    test_path,
    target_size=(44, 44),
    color_mode="grayscale",
    batch_size=32,
    class_mode='categorical'
)
model.evaluate(test_gen)
# print(f"Test Accuracy: {test_acc}")

Found 2496 images belonging to 7 classes.
78/78 ━━━━━━━━━━━━━━━━━━━━ 16s 206ms/step - accuracy: 0.9797 - loss: 0.1590


[0.19974620640277863, 0.9751602411270142]

In [13]:
model.save(f"./chess_FEN_model.h5")

In [14]:
tfjs.converters.save_keras_model(model, "chess_FEN_model")

failed to lookup keras version from the file,
    this is likely a weight only file


In [15]:
!tensorflowjs_converter --input_format keras ./ghaly_chess_FEN_model.h5 ./my_model_as_tfjs

2025-02-15 03:33:04.345490: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739583184.461098   32539 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739583184.467793   32539 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
🌲 Try https://ydf.readthedocs.io, the successor of TensorFlow Decision Forests with more features and faster training!
failed to lookup keras version from the file,
    this is likely a weight only file
